I used Ollama to run the LLM (download from https://ollama.com/ and run exe)
pip install Ollama
ollama pull llama3.1

the rest is done in a virtual env.



In [92]:
%pip install langchain langchain_community pypdf docarray pydantic==1.10.8

Note: you may need to restart the kernel to use updated packages.


In [93]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

MODEL = 'llama3.1'
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [94]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [95]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("./docs/I-90_Instructions.pdf")
pages = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50).split_documents(loader.load())
pages

[Document(metadata={'source': './docs/I-90_Instructions.pdf', 'page': 0}, page_content='Fee Rule\nDRAFT\n10/03/2023\nInstructions for Application to Replace\nPermanent Resident Card \nDepartment of Homeland Security\nU.S. Citizenship and Immigration ServicesUSCIS\n Form I-90\nOMB No. 1615-0082\nExpires 02/28/2027\nForm I-90 Instructions   04/01/24 Page 1 of 11What Is the Purpose of Form I-90?\nThis application is used by lawful permanent residents and permanent residents in commuter status to apply for \nreplacement or renewal of existing Permanent Resident Cards.  Conditional permanent residents may also use this application to apply for replacement of an existing Permanent Resident Card.  Conditional permanent residents may not \nuse this application to replace, for any reason, an existing Permanent Resident Card that is expired or will expire within  \n90 days.\nNOTE:  Conditional permanent residents (for example, CR1, CR2, CF1, CF2) who obtained their status through marriage \nor e

In [96]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based only on the context below.
If you can't answer the question, reply "I can't find any information about that".
Do not discuss topics outside of the context.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [97]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
)

In [101]:
retriever = vectorstore.as_retriever()
retriever.invoke("What is an I-90?")

[Document(metadata={'source': './docs/I-90_Instructions.pdf', 'page': 1}, page_content='Fee Rule\nDRAFT\n10/03/2023\nForm I-90 Instructions   04/01/24 Page 2 of 11\nEvidence.  When you file your application, you must submit all evidence and supporting documents listed in the Specific \nInstructions sections of these Instructions.\nBiometric Services Appointment.  USCIS may require you to appear for an interview or provide biometrics (fingerprints, \nphotograph, and/or signature) at any time to verify your identity , obtain additional information, and conduct background \nand security checks, including a check of criminal history records maintained by the Federal Bureau of Investigation \n(FBI), before making a decision on your application or petition.  If we determine that a biometric services appointment \nis necessary, we will send you an appointment notice with the date, time, and location of your appointment.  If you are currently overseas, your notice will instruct you to contact 

In [99]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [100]:
questions = [
    "What is a valid signature?",
    "What is form I-751?",
    "What is a POE?",
    "Where can I check my case status?"
]

for i in questions:
    print(f"Questions: {i}")
    print(f"Answer: {chain.invoke({'question': i})}")
    print()

Questions: What is a valid signature?
Answer: According to the text, a valid signature on Form I-90 can be:

* An original handwritten ink signature
* A photocopied copy of an original handwritten ink signature
* A faxed copy of an original handwritten ink signature
* A scanned copy of an original handwritten ink signature

In other words, a valid signature is one that is the original handwritten ink signature or a photocopy/fax/scanned copy of it.

Questions: What is form I-751?
Answer: The text does not mention Form I-751. However, it discusses various reasons and instructions for filing Form I-90, which is a petition for a replacement Permanent Resident Card (also known as a Green Card).

Questions: What is a POE?
Answer: According to the provided document, a "POE" stands for Port of Entry, which refers to an official point of entry into the United States. In the context of Form I-90 instructions, it's mentioned that USCIS will send the Permanent Resident Card to the appropriate Por